<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Метод-TF-IDF" data-toc-modified-id="Метод-TF-IDF-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Метод TF-IDF</a></span></li><li><span><a href="#Эмбеддинг-BERT" data-toc-modified-id="Эмбеддинг-BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Эмбеддинг BERT</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [72]:
#подключим необходимые для работы библиотеки

import numpy as np
import pandas as pd

import warnings
warnings.simplefilter("ignore")
pd.options.display.float_format = '{:,.2f}'.format

import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.utils import shuffle
from tqdm import notebook 
import re
import spacy

In [66]:
#прочитаем исходный датафрейм

try:
    data = pd.read_csv('C:/Users/IshkulovIM/pythonworks/yandex_practikum/toxic_comments.csv', sep=',') 
    
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', sep=',')

In [67]:
# изучим основную информацию о датафрейме

print('Первые 10 строк датафрейма:')
display(data.head(10))
print('Основная информация о датафрейме:')
display(data.info())
print('Размер датасета:')
display(data.shape)
print('Доля пропущенных данных в датафрейме:')
display(data.isna().mean())
print('Количество дубликатов: ')
display(data.duplicated().sum())

Первые 10 строк датафрейма:


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Основная информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

Размер датасета:


(159571, 2)

Доля пропущенных данных в датафрейме:


text    0.00
toxic   0.00
dtype: float64

Количество дубликатов: 


0

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Для начала исследуем баланс классов.
</div>

In [68]:
#изучим количество значений каждого класса

print(data['toxic'].value_counts())

0    143346
1     16225
Name: toxic, dtype: int64


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Видим сильный дисбаланс классов, учтем этот факт при обучении моделей. Далее проведем лемматизацию текстов и очистку текста от ненужных символов.
</div>

In [76]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [116]:
#напишем функции для очистки и лемматизации исходного текста

corpus = data['text'].values

def lemmatize(text):
    lemm_list = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in lemm_list])
    return lemm_text

def clear_text(text):
    clear_list = re.sub(r'[^a-zA-Z]',' ', text)
    clear_text = clear_list.split() 
    clear_text_1 = " ".join(clear_text)
    return clear_text_1

data['lemm_text'] = data['text'].apply(lemmatize).apply(clear_text)

In [119]:
#далее разделим общую выборку на обучающую и тестовую

target_tf = data['toxic'].values

features_tf = data['lemm_text']

features_train_tf, features_test_tf, target_train_tf, target_test_tf = train_test_split(features_tf, target_tf, 
                                                                                        test_size=0.25, random_state=12345)

#далее узнаем размеры выборок

print('Размер обучающей выборки равен: ', features_train_tf.shape)
print('Размер тестовой выборки равен: ', features_test_tf.shape)

Размер обучающей выборки равен:  (119678,)
Размер тестовой выборки равен:  (39893,)


In [120]:
#вычислим TF-IDF для корпуса текстов

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

corpus_train = features_train_tf.values
corpus_test = features_test_tf.values

count_tf_idf = TfidfVectorizer() 

tf_idf_train = count_tf_idf.fit(corpus_train) 
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\IshkulovIM\Anaconda3\lib\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Далее перейдем к обучению моделей. Используем метод векторизации текстов TF-IDF, а также эмбеддинг BERT.
</div>

## Обучение

### Метод TF-IDF

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Перейдем к обучению моделей. Начнем с модели "Дерево решений".
</div>

In [121]:
%%time
#зададим гиперпараметры дерева решений

max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

max_features = [int(x) for x in np.linspace(1, 20, num = 10)]

class_weight = ['balanced']

dt_grid = {'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
           'max_features': max_features,
           'class_weight': class_weight
           }

model_dt_tf = GridSearchCV(DecisionTreeClassifier(random_state=12345), dt_grid, scoring = 'f1_weighted')
# обучаем модель
model_dt_tf.fit(tf_idf_train, target_train_tf)

Wall time: 15min 14s


GridSearchCV(estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'class_weight': ['balanced'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_features': [1, 3, 5, 7, 9, 11, 13, 15, 17, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_weighted')

In [122]:
#определяем лучшие гиперпараметры для дерева решений

print('Наивысшая оценка: %s' % model_dt_tf.best_score_)
print('Лучшие гиперпараметры: %s' % model_dt_tf.best_params_)

Наивысшая оценка: 0.8504495782437017
Лучшие гиперпараметры: {'class_weight': 'balanced', 'max_depth': 10, 'max_features': 7, 'min_samples_leaf': 1, 'min_samples_split': 5}


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Далее рассмотрим модель случайного леса.
</div>

In [123]:
%%time
#зададим гиперпараметры случайного леса

n_estimators = [3, 5, 7, 10]

max_depth = [3, 5, 10]

max_features = ['auto']

min_samples_split = [5, 10]

min_samples_leaf = [2, 4]

class_weight = ['balanced']

rf_grid = {'n_estimators': n_estimators,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
           'class_weight': class_weight
           }

model_rf_tf = GridSearchCV(RandomForestClassifier(random_state=12345), rf_grid, scoring = 'f1_weighted')
# обучаем модель
model_rf_tf.fit(tf_idf_train, target_train_tf)

Wall time: 4min 54s


GridSearchCV(estimator=RandomForestClassifier(random_state=12345),
             param_grid={'class_weight': ['balanced'], 'max_depth': [3, 5, 10],
                         'max_features': ['auto'], 'min_samples_leaf': [2, 4],
                         'min_samples_split': [5, 10],
                         'n_estimators': [3, 5, 7, 10]},
             scoring='f1_weighted')

In [124]:
#определяем лучшие гиперпараметры для случайного леса

print('Наивысшая оценка: %s' % model_rf_tf.best_score_)
print('Лучшие гиперпараметры: %s' % model_rf_tf.best_params_)

Наивысшая оценка: 0.710972033787496
Лучшие гиперпараметры: {'class_weight': 'balanced', 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 10}


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Перейдем к модели "Логистическая регрессия".
</div>

In [125]:
%%time
#зададим гиперпараметры логистической регрессии

C = [int(x) for x in np.linspace(start = 1, stop = 10, num = 11)]

class_weight = ['balanced']

lr_grid = {'C': C,
           'class_weight': class_weight}


model_lr_tf = GridSearchCV(LogisticRegression(random_state=12345), lr_grid, scoring = 'f1_weighted')
# обучаем модель
model_lr_tf.fit(tf_idf_train, target_train_tf)

Wall time: 4min 35s


GridSearchCV(estimator=LogisticRegression(random_state=12345),
             param_grid={'C': [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'class_weight': ['balanced']},
             scoring='f1_weighted')

In [126]:
#определяем лучшие гиперпараметры для логистической регрессии

print('Наивысшая оценка: %s' % model_lr_tf.best_score_)
print('Лучшие гиперпараметры: %s' % model_lr_tf.best_params_)

Наивысшая оценка: 0.9523331700129818
Лучшие гиперпараметры: {'C': 10, 'class_weight': 'balanced'}


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Далее обучим модель бустинга CatBoostRegressor.
</div>

In [127]:
%%time
#зададим гиперпараметры для CatBoostRegressor

n_estimators = [10, 30]
depth = [1, 3, 5]
auto_class_weights = ['Balanced']

cb_grid = {'n_estimators': n_estimators,
           'depth': depth, 
           'auto_class_weights': auto_class_weights}

model_cb_tf =  GridSearchCV(CatBoostClassifier(random_seed=12345), cb_grid, scoring = 'f1_weighted')
# обучаем модель
model_cb_tf.fit(tf_idf_train, target_train_tf)

Learning rate set to 0.5
0:	learn: 0.6333768	total: 94.8ms	remaining: 854ms
1:	learn: 0.6066659	total: 174ms	remaining: 696ms
2:	learn: 0.5927194	total: 254ms	remaining: 593ms
3:	learn: 0.5803730	total: 335ms	remaining: 503ms
4:	learn: 0.5687940	total: 417ms	remaining: 417ms
5:	learn: 0.5610820	total: 497ms	remaining: 332ms
6:	learn: 0.5451794	total: 577ms	remaining: 247ms
7:	learn: 0.5343972	total: 658ms	remaining: 164ms
8:	learn: 0.5241749	total: 738ms	remaining: 82ms
9:	learn: 0.5118098	total: 818ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6325757	total: 112ms	remaining: 1.01s
1:	learn: 0.6068546	total: 186ms	remaining: 744ms
2:	learn: 0.5931161	total: 262ms	remaining: 612ms
3:	learn: 0.5813125	total: 339ms	remaining: 509ms
4:	learn: 0.5695818	total: 415ms	remaining: 415ms
5:	learn: 0.5527790	total: 492ms	remaining: 328ms
6:	learn: 0.5420636	total: 570ms	remaining: 244ms
7:	learn: 0.5297552	total: 647ms	remaining: 162ms
8:	learn: 0.5154703	total: 725ms	remaining: 80.5ms
9

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x000002A5BBB31940>,
             param_grid={'auto_class_weights': ['Balanced'], 'depth': [1, 3, 5],
                         'n_estimators': [10, 30]},
             scoring='f1_weighted')

In [128]:
#определяем лучшие гиперпараметры для CatBoostRegressor

print('Наивысшая оценка: %s' % model_cb_tf.best_score_)
print('Лучшие гиперпараметры: %s' % model_cb_tf.best_params_)

Наивысшая оценка: 0.9217559638968158
Лучшие гиперпараметры: {'auto_class_weights': 'Balanced', 'depth': 5, 'n_estimators': 30}


###  Эмбеддинг BERT

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Обучим модель, используя эмбеддинг BERT. Для ускорения обучения буду подавать в эмбеддинг только часть данных (10 часть от исходного датафрейма). Понимаю, что тестовая выборка будет отличаться по размеру. Но просто было интересно попробовать данный споосб и посмотреть на его метрики качества.
</div>

In [ ]:
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12_pt.tar.gz

In [ ]:
!tar -xvzf multi_cased_L-12_H-768_A-12_pt.tar.gz

In [25]:
data = data.sample(15900).reset_index(drop=True) 

data.shape

(15900, 3)

In [26]:
# инициализируем токенизатор

tokenizer = transformers.BertTokenizer(vocab_file='vocab.txt')
tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

# токенизируем текст

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
        
# создадим маску для важных токенов     
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape) 

(15900, 512)


In [27]:
config = transformers.BertConfig.from_json_file('bert_config.json')
model = transformers.BertModel.from_pretrained('pytorch_model.bin', config=config)
batch_size = 100

# сделаем пустой список для хранения эмбеддингов твитов
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    
    # преобразуем данные
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    # преобразуем маску
    attention_mask_batch = torch.LongTensor(
                            attention_mask[batch_size*i:batch_size*(i+1)])
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    # преобразуем элементы методом numpy() к типу numpy.array
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

Some weights of the model checkpoint at pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [129]:
del batch
del attention_mask_batch
del batch_embeddings

In [28]:
#определим признаки и целевой признак

features = np.concatenate(embeddings)
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.5, 
                                                                              random_state=12345)


#далее узнаем размеры выборок

print('Размер обучающей выборки равен: ', features_train.shape)
print('Размер тестовой выборки равен: ', features_test.shape)

Размер обучающей выборки равен:  (7950, 768)
Размер тестовой выборки равен:  (7950, 768)


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Перейдем к обучению моделей. Начнем с модели "Дерево решений".
</div>

In [29]:
%%time
#зададим гиперпараметры дерева решений

max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

max_features = [int(x) for x in np.linspace(1, 20, num = 10)]

class_weight = ['balanced']

dt_grid = {'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
           'max_features': max_features,
           'class_weight': class_weight
           }

model_dt = GridSearchCV(DecisionTreeClassifier(random_state=12345), dt_grid, scoring = 'f1_weighted')
# обучаем модель
model_dt.fit(features_train, target_train)

Wall time: 4min 7s


GridSearchCV(estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'class_weight': ['balanced'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_features': [1, 3, 5, 7, 9, 11, 13, 15, 17, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_weighted')

In [30]:
#определяем лучшие гиперпараметры для дерева решений

print('Наивысшая оценка: %s' % model_dt.best_score_)
print('Лучшие гиперпараметры: %s' % model_dt.best_params_)

Наивысшая оценка: 0.8387974773987628
Лучшие гиперпараметры: {'class_weight': 'balanced', 'max_depth': 4, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Далее рассмотрим модель случайного леса.
</div>

In [31]:
%%time
#зададим гиперпараметры случайного леса

n_estimators = [3, 5, 7, 10]

max_depth = [3, 5, 10]

max_features = ['auto']

min_samples_split = [5, 10]

min_samples_leaf = [2, 4]

class_weight = ['balanced']

rf_grid = {'n_estimators': n_estimators,
           'max_features': max_features,
           'max_depth': max_depth,
           'min_samples_split': min_samples_split,
           'min_samples_leaf': min_samples_leaf,
           'class_weight': class_weight
           }

model_rf = GridSearchCV(RandomForestClassifier(random_state=12345), rf_grid, scoring = 'f1_weighted')
# обучаем модель
model_rf.fit(features_train,target_train)

Wall time: 1min 29s


GridSearchCV(estimator=RandomForestClassifier(random_state=12345),
             param_grid={'class_weight': ['balanced'], 'max_depth': [3, 5, 10],
                         'max_features': ['auto'], 'min_samples_leaf': [2, 4],
                         'min_samples_split': [5, 10],
                         'n_estimators': [3, 5, 7, 10]},
             scoring='f1_weighted')

In [32]:
#определяем лучшие гиперпараметры для случайного леса

print('Наивысшая оценка: %s' % model_rf.best_score_)
print('Лучшие гиперпараметры: %s' % model_rf.best_params_)

Наивысшая оценка: 0.9023578852278231
Лучшие гиперпараметры: {'class_weight': 'balanced', 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 10}


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Перейдем к модели "Логистическая регрессия".
</div>

In [33]:
%%time
#зададим гиперпараметры логистической регрессии

C = [int(x) for x in np.linspace(start = 1, stop = 10, num = 11)]

class_weight = ['balanced']

lr_grid = {'C': C,
           'class_weight': class_weight}

model_lr = GridSearchCV(LogisticRegression(random_state=12345), lr_grid, scoring = 'f1_weighted')
# обучаем модель
model_lr.fit(features_train,target_train)

Wall time: 13.8 s


GridSearchCV(estimator=LogisticRegression(random_state=12345),
             param_grid={'C': [1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'class_weight': ['balanced']},
             scoring='f1_weighted')

In [34]:
#определяем лучшие гиперпараметры для логистической регрессии

print('Наивысшая оценка: %s' % model_lr.best_score_)
print('Лучшие гиперпараметры: %s' % model_lr.best_params_)

Наивысшая оценка: 0.8878633842628678
Лучшие гиперпараметры: {'C': 7, 'class_weight': 'balanced'}


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Далее обучим модель бустинга CatBoostRegressor.
</div>

In [35]:
%%time
#зададим гиперпараметры для CatBoostRegressor

n_estimators = [10, 30, 50, 70, 100]
depth = [1, 3, 5, 7, 10]
auto_class_weights = ['Balanced']

cb_grid = {'n_estimators': n_estimators,
           'depth': depth, 
           'auto_class_weights': auto_class_weights}

model_cb =  GridSearchCV(CatBoostClassifier(random_seed=12345), cb_grid, scoring = 'f1_weighted')
# обучаем модель
model_cb.fit(features_train,target_train)

Learning rate set to 0.5
0:	learn: 0.6319027	total: 9.16ms	remaining: 82.5ms
1:	learn: 0.5988512	total: 12.2ms	remaining: 49ms
2:	learn: 0.5805116	total: 15.1ms	remaining: 35.2ms
3:	learn: 0.5611910	total: 18.1ms	remaining: 27.2ms
4:	learn: 0.5469343	total: 21.2ms	remaining: 21.2ms
5:	learn: 0.5384142	total: 24.3ms	remaining: 16.2ms
6:	learn: 0.5332503	total: 27.3ms	remaining: 11.7ms
7:	learn: 0.5253013	total: 30.5ms	remaining: 7.63ms
8:	learn: 0.5185238	total: 33.7ms	remaining: 3.74ms
9:	learn: 0.5098552	total: 36.7ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6290932	total: 9.69ms	remaining: 87.2ms
1:	learn: 0.6035742	total: 14.9ms	remaining: 59.6ms
2:	learn: 0.5858373	total: 18.3ms	remaining: 42.7ms
3:	learn: 0.5699823	total: 21.6ms	remaining: 32.4ms
4:	learn: 0.5565446	total: 24.9ms	remaining: 24.9ms
5:	learn: 0.5447040	total: 27.8ms	remaining: 18.5ms
6:	learn: 0.5317864	total: 30.8ms	remaining: 13.2ms
7:	learn: 0.5249453	total: 33.8ms	remaining: 8.45ms
8:	learn: 0.5110501

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x0000029F82247910>,
             param_grid={'auto_class_weights': ['Balanced'],
                         'depth': [1, 3, 5, 7, 10],
                         'n_estimators': [10, 30, 50, 70, 100]},
             scoring='f1_weighted')

In [36]:
#определяем лучшие гиперпараметры для CatBoostRegressor

print('Наивысшая оценка: %s' % model_cb.best_score_)
print('Лучшие гиперпараметры: %s' % model_cb.best_params_)

Наивысшая оценка: 0.9161063076470126
Лучшие гиперпараметры: {'auto_class_weights': 'Balanced', 'depth': 7, 'n_estimators': 100}


In [130]:
#составим таблицу для сравнения полученных результатов

compare_of_models = pd.DataFrame([['15min 14s', '0.85', '4min 7s', '0.84'],
                                  ['4min 54s', '0.71', '1min 29s', '0.90'],  
                                  ['4min 35s', '0.95', '13.8 s', '0.89'],
                                  ['5min 56s', '0.92', '33min 12s', '0.92']],
                                   index=['Дерево решений', 'Случайный лес', 'Логистическая регрессия', 'CatBoostRegressor'],
                                   columns=[['Метод TF-IDF','Метод TF-IDF', 'Эмбеддинг BERT', 'Эмбеддинг BERT'],
                                            ['Время обучения модели','Метрика F1 на обучающей выборке',
                                             'Время обучения модели','Метрика F1 на обучающей выборке']])
                                 
compare_of_models

Метод TF-IDF                                  \
                        Время обучения модели Метрика F1 на обучающей выборке   
Дерево решений                      15min 14s                            0.85   
Случайный лес                        4min 54s                            0.71   
Логистическая регрессия              4min 35s                            0.95   
CatBoostRegressor                    5min 56s                            0.92   

                               Эмбеддинг BERT                                  
                        Время обучения модели Метрика F1 на обучающей выборке  
Дерево решений                        4min 7s                            0.84  
Случайный лес                        1min 29s                            0.90  
Логистическая регрессия                13.8 s                            0.89  
CatBoostRegressor                   33min 12s                            0.92

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Как можно заметить по таблице выше, модели, обученные на эмбеддинге BERT, показали результаты сравнимые с методом TF-IDF, при этом было затрачено меньше времени на обучение. Выберем модель случайного леса для получения предсказаний на тестовой выборке, т.к. метрика f1 достаточно высока на тренировочной выборке, при этом время обучения достаточно небольшое.
</div>

## Выводы

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Получим результаты модели случайного леса на тестовой модели.
</div>

In [49]:
%%time 

#получение предсказания модели случайного леса на тестовой выборке

predicted_rf_test = model_rf.predict(features_test)

print('Метрика качества f1-score модели случайного леса на тестовой выборке:{:.2f}'.
      format(f1_score(predicted_rf_test, target_test, average='weighted')))

Метрика качества f1-score модели случайного леса на тестовой выборке:0.91
Wall time: 40 ms


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Получили значение метрики f1 выше 0.75, что говорит о высокой способности модели давать качественные предсказания.
</div>